In [ ]:
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from utils import get_data, get_batches
import numpy as np

In [ ]:
DATASET_NAME = "sberquad"
DEVICE = "cpu"

In [ ]:
dataset = load_dataset(DATASET_NAME)
test_data = dataset["test"]

tokenizer = AutoTokenizer.from_pretrained("./tokenizer_rubert-tiny2")
model = AutoModel.from_pretrained("./new_rubert-tiny2").to(DEVICE)
model.eval()

queries, passages = get_data(range(len(test_data)), test_data)
test_data_batched = get_batches(queries, passages, 400)

In [ ]:
def get_ndcg(scores, labels):
    sorted_indices = torch.argsort(scores, descending=True)
    sorted_labels = labels[sorted_indices]

    dcg = sum((sorted_labels[i].item() / np.log2(i + 2)) for i in range(len(sorted_labels)))
    idcg = sum((1 / np.log2(i + 2)) for i in range(sum(labels).int().item()))

    return dcg / idcg

In [ ]:
ndcg_scores = []
progressBar = tqdm(range(len(test_data_batched)))

for batch in test_data_batched:
    
    query = tokenizer(batch["question"], return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    passage = tokenizer(batch["context"], return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    
    with torch.no_grad():
        query_emb = model(**query).last_hidden_state.mean(dim=1)
        passage_emb = model(**passage).last_hidden_state.mean(dim=1)
    

    scores = torch.zeros(len(query_emb), len(passage_emb))
    for i in range(len(query_emb)):
        scores[i] = F.cosine_similarity(query_emb[i].unsqueeze(0), passage_emb)

    for i in range(len(batch["question"])):
        labels = torch.zeros(len(batch["context"]))
        labels[i] = 1
        ndcg_scores.append(get_ndcg(scores[i], labels))

    torch.cuda.empty_cache()
    progressBar.update(1)

print(np.mean(ndcg_scores))